## Co-occurencier 
Se munir du corpus de travail

Ici deux méthodes sont développées et à prendre en compte

1) Le corpus sous format .txt ne comprend pas d'informations concernant la pagination, il faut alors ignorer ce point et se concentrer sur un positionnement relatif des termes par rapport à la séance en entier
2) Le corpus est numérisé mais pas encore océriser, il est alors assez facile de rajouter un petit marqueur spécial indiquant un changement de page (ex : "===chgpage===") qu'il est ensuite possible de détecter lors de l'ouverture du documents et nous permettrait plus facilement de retrouver cette information

L'ORC le plus performant étant ici celui sans la pagination indiquée, le travail fut réaliser en suivant la première méthode et en indiquant néanmoins la date de la séance ainsi que la position relative de l'extrait

Fournir deux listes de mots (ou utiliser celles fournies de base), listes pouvant être modifiées
Il est également possible de travailler sur une unique liste afin de déterminer par exemple si les mots d'un même champ lexical sont employés dans les mêmes extraits ou non.

Il est suggéré dans un premier temps d'utiliser le notebook recherchemotclefs.ipynb sur le corpus d'entrée afin de ne concerver uniquement les documents qui ont au moins une occurence des mots souhaités et ne pas travailler sur l'ensemble du corpus qui ne correspond pas forcément.


In [ ]:
import os
import re
import pandas as pd
from collections import defaultdict
from tqdm import tqdm


DOSSIER_TEXTES = "C:/Users/portable_laura/Documents/coursM1/Mémoire/Sources/ocr_sorted"      
LISTE1_PATH = "liste1.txt"
LISTE2_PATH = "liste2.txt"
CONTEXT_SIZE = 10                    

def lire_liste(path):
    with open(path, "r", encoding="utf-8") as f:
        return [mot.strip().lower() for mot in f if mot.strip()]

def decouper_pages(texte):
    texte = texte.lower()
    texte = re.sub(r"[^\w\s]", "", texte) 
    return texte.split()

def chercher_cooccurrences(tokens, liste1, liste2, doc, page_num, context_size=10):
    resultats = []

    for i, mot in enumerate(tokens):
        if mot in liste1:
            contexte_debut = max(0, i - context_size)
            contexte_fin = min(len(tokens), i + context_size + 1)
            contexte = tokens[contexte_debut:contexte_fin]
            
            for j, mot2 in enumerate(contexte):
                if mot2 in liste2:
                    if mot != mot2:
                        distance = abs(i - (contexte_debut + j))
                        extrait_contexte = " ".join(contexte)
                        resultats.append({
                            "mot1": mot,
                            "mot2": mot2,
                            "distance": distance,
                            "contexte": extrait_contexte,
                            "document": doc,
                            "page": page_num + 1
                        })
    return resultats

#liste1 = lire_liste(LISTE1_PATH)
#liste2 = lire_liste(LISTE2_PATH)
liste1 =["freycinet", "gabarit", "Tardieu",
              "halage", "curage", "dragage", "transbordement", "cabotage",
              "irrigation", "adduction", "drainage", "canalisée", "mouillage", "canalisation"
              "péniche",  'bateau', "bateaux", "gabare", "allège", "convoi", "automoteur", "chaland", 
              'remorqueur', "bac", "fret", "tonnage",
              "tirant","écluse", 'fluvial', "fleuve", "rivière", "affluent","riviere", "canal", "canaux",
              "bassin", "versant", "flux",
              'batellerie', 'navigable',"navigation", "navigabilité", "pilotage",
              "transport", "houille",  "marchandise", 
              "eau", "inondation", "barrage",  "chantier",
              "loire", "oise", "garonne", "rhin", "rhône", "seine", "midi","marne",
              "ascenseur", "embarcation", "port", "quai",
              "ponts", "chaussees"]

liste2 = ["travaux", "investissement", "publics", "finances", "credit", "dotation", "budget", "dépense","tresor","utilite",
               "amenagement","infrastructure","concession","maitrise","ouvrage","questeur","trésorier","compagnie"]


cooc_finales = []

for fichier in tqdm(os.listdir(DOSSIER_TEXTES),desc="documents à ouvrir"):
    if not fichier.endswith(".txt"):
        continue

    chemin_complet = os.path.join(DOSSIER_TEXTES, fichier)
    with open(chemin_complet, "r", encoding="utf-8") as f:
        texte = f.read()

    pages = decouper_pages(texte)

    for num_page, page in enumerate(pages):
        tokens = decouper_pages(page)
        cooc_page = chercher_cooccurrences(tokens, liste1, liste2, fichier, num_page, CONTEXT_SIZE)
        cooc_finales.extend(cooc_page)


df = pd.DataFrame(cooc_finales)
df = df.drop_duplicates(subset=["mot1", "mot2", "document", "page", "contexte"])
df.to_csv("C:/Users/portable_laura/Desktop/memoire_prod_finale/data/cooccurrences.csv", index=False, encoding="utf-8")

print(f"{len(df)} cooccurrences trouvées. Résultat exporté dans 'cooccurrences.csv'")


documents à ouvrir: 100%|██████████| 7461/7461 [22:05<00:00,  5.63it/s]  

0 cooccurrences trouvées. Résultat exporté dans 'cooccurrences.csv'


In [3]:
df

""
